<a href="https://colab.research.google.com/github/vineetbiyani09/NLP/blob/master/NLP_10_Topic_Modelling_Newsdata_J010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/labnlpabcnews-date-text.csv')
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [ ]:
len(documents)

1226258

In [ ]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


### Data Preprocessing

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

#### Lemmatize example

In [ ]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


#### Stemmer Example

In [ ]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [ ]:
processed_docs = documents['headline_text'].map(preprocess)

In [ ]:
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

### Bag of words on the dataset

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3570, 1), (3571, 1)]

In [ ]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 838 ("want") appears 1 time.
Word 3570 ("compulsori") appears 1 time.
Word 3571 ("ratepay") appears 1 time.


### TF-IDF

In [ ]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [ ]:
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5842699484464488),
 (1, 0.38798859072167835),
 (2, 0.5008422243250992),
 (3, 0.5071987254965034)]


### Running LDA using Bag of Words

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.017*"border" + 0.016*"australia" + 0.013*"interview" + 0.012*"health" + 0.010*"talk" + 0.010*"say" + 0.009*"coronavirus" + 0.009*"quarantin" + 0.009*"weather" + 0.008*"mental"
Topic: 1 
Words: 0.045*"trump" + 0.029*"charg" + 0.029*"case" + 0.028*"court" + 0.024*"murder" + 0.019*"face" + 0.018*"alleg" + 0.015*"jail" + 0.015*"woman" + 0.014*"accus"
Topic: 2 
Words: 0.044*"coronavirus" + 0.034*"covid" + 0.015*"tasmania" + 0.014*"school" + 0.014*"govern" + 0.012*"dead" + 0.011*"fund" + 0.011*"communiti" + 0.010*"student" + 0.010*"break"
Topic: 3 
Words: 0.039*"queensland" + 0.031*"victoria" + 0.027*"melbourn" + 0.020*"coast" + 0.014*"farmer" + 0.014*"australia" + 0.014*"coronavirus" + 0.013*"win" + 0.012*"gold" + 0.012*"west"
Topic: 4 
Words: 0.022*"home" + 0.022*"kill" + 0.022*"news" + 0.018*"state" + 0.015*"perth" + 0.014*"peopl" + 0.014*"bushfir" + 0.013*"attack" + 0.012*"child" + 0.011*"care"
Topic: 5 
Words: 0.020*"open" + 0.020*"adelaid" + 0.016*"final" + 0.015*"mi

Cool! Can you distinguish different topics using the words in each topic and their corresponding weights?

### Running LDA using TF-IDF

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.014*"drum" + 0.009*"friday" + 0.009*"thursday" + 0.008*"david" + 0.007*"hill" + 0.007*"march" + 0.005*"hong" + 0.005*"georg" + 0.005*"kong" + 0.005*"histori"
Topic: 1 Word: 0.008*"final" + 0.007*"scott" + 0.007*"wall" + 0.007*"world" + 0.006*"juli" + 0.006*"australia" + 0.006*"cricket" + 0.006*"leagu" + 0.005*"rugbi" + 0.005*"street"
Topic: 2 Word: 0.011*"govern" + 0.009*"health" + 0.009*"countri" + 0.008*"coronavirus" + 0.008*"elect" + 0.007*"hour" + 0.006*"budget" + 0.006*"fund" + 0.006*"feder" + 0.006*"say"
Topic: 3 Word: 0.016*"interview" + 0.007*"extend" + 0.006*"jam" + 0.005*"asylum" + 0.005*"mount" + 0.005*"john" + 0.005*"seeker" + 0.004*"phone" + 0.004*"love" + 0.004*"accid"
Topic: 4 Word: 0.012*"polic" + 0.009*"search" + 0.008*"miss" + 0.008*"coast" + 0.008*"woman" + 0.007*"beach" + 0.006*"peter" + 0.006*"sydney" + 0.006*"bodi" + 0.006*"alan"
Topic: 5 Word: 0.022*"trump" + 0.013*"murder" + 0.012*"charg" + 0.010*"polic" + 0.010*"court" + 0.009*"alleg" + 0.008*"

### Classification of the topics

### Performance evaluation by classifying sample document using LDA Bag of Words model

In [ ]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [ ]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8874959349632263	 
Topic: 0.028*"elect" + 0.021*"say" + 0.014*"govern" + 0.012*"labor" + 0.011*"chang" + 0.011*"speak" + 0.010*"drum" + 0.010*"andrew" + 0.009*"public" + 0.009*"vote"

Score: 0.012502622790634632	 
Topic: 0.044*"coronavirus" + 0.034*"covid" + 0.015*"tasmania" + 0.014*"school" + 0.014*"govern" + 0.012*"dead" + 0.011*"fund" + 0.011*"communiti" + 0.010*"student" + 0.010*"break"

Score: 0.012500757351517677	 
Topic: 0.037*"coronavirus" + 0.019*"south" + 0.017*"north" + 0.017*"year" + 0.016*"restrict" + 0.016*"rise" + 0.015*"nation" + 0.015*"water" + 0.015*"victorian" + 0.012*"price"

Score: 0.012500663287937641	 
Topic: 0.038*"australia" + 0.021*"world" + 0.019*"market" + 0.017*"protest" + 0.017*"record" + 0.015*"australian" + 0.014*"busi" + 0.014*"china" + 0.012*"time" + 0.011*"coronavirus"

Score: 0.012500016018748283	 
Topic: 0.039*"queensland" + 0.031*"victoria" + 0.027*"melbourn" + 0.020*"coast" + 0.014*"farmer" + 0.014*"australia" + 0.014*"coronavirus" + 0.0

Our test document has the highest probability to be part of the topic on the top.

### Performance evaluation by classifying sample document using LDA TF-IDF model

In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8874947428703308	 
Topic: 0.011*"govern" + 0.009*"health" + 0.009*"countri" + 0.008*"coronavirus" + 0.008*"elect" + 0.007*"hour" + 0.006*"budget" + 0.006*"fund" + 0.006*"feder" + 0.006*"say"

Score: 0.012501240707933903	 
Topic: 0.019*"news" + 0.016*"market" + 0.012*"rural" + 0.008*"weather" + 0.007*"christma" + 0.007*"michael" + 0.007*"price" + 0.006*"nation" + 0.006*"australian" + 0.006*"farmer"

Score: 0.012500806711614132	 
Topic: 0.010*"coronavirus" + 0.008*"australia" + 0.008*"wednesday" + 0.007*"updat" + 0.007*"live" + 0.007*"covid" + 0.006*"china" + 0.006*"korea" + 0.006*"august" + 0.006*"south"

Score: 0.012500768527388573	 
Topic: 0.018*"coronavirus" + 0.011*"victoria" + 0.010*"covid" + 0.008*"tuesday" + 0.007*"queensland" + 0.006*"age" + 0.006*"bushfir" + 0.006*"lockdown" + 0.006*"pandem" + 0.006*"rain"

Score: 0.012500687502324581	 
Topic: 0.014*"drum" + 0.009*"friday" + 0.009*"thursday" + 0.008*"david" + 0.007*"hill" + 0.007*"march" + 0.005*"hong" + 0.005*"georg"

Our test document has the highest probability to be part of the topic on the top.

### Testing model on unseen document

In [ ]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3500000536441803	 Topic: 0.038*"australia" + 0.021*"world" + 0.019*"market" + 0.017*"protest" + 0.017*"record"
Score: 0.1833333671092987	 Topic: 0.017*"border" + 0.016*"australia" + 0.013*"interview" + 0.012*"health" + 0.010*"talk"
Score: 0.18333333730697632	 Topic: 0.039*"queensland" + 0.031*"victoria" + 0.027*"melbourn" + 0.020*"coast" + 0.014*"farmer"
Score: 0.18333324790000916	 Topic: 0.028*"elect" + 0.021*"say" + 0.014*"govern" + 0.012*"labor" + 0.011*"chang"
Score: 0.01666666753590107	 Topic: 0.045*"trump" + 0.029*"charg" + 0.029*"case" + 0.028*"court" + 0.024*"murder"
Score: 0.01666666753590107	 Topic: 0.044*"coronavirus" + 0.034*"covid" + 0.015*"tasmania" + 0.014*"school" + 0.014*"govern"
Score: 0.01666666753590107	 Topic: 0.022*"home" + 0.022*"kill" + 0.022*"news" + 0.018*"state" + 0.015*"perth"
Score: 0.01666666753590107	 Topic: 0.020*"open" + 0.020*"adelaid" + 0.016*"final" + 0.015*"miss" + 0.015*"trial"
Score: 0.01666666753590107	 Topic: 0.058*"polic" + 0.026*"dona